In [2]:
import pandas as pd
import numpy as np
from supervised.automl import AutoML
from dateutil.relativedelta import relativedelta
from sklearn.metrics import roc_auc_score

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## DF Original

In [3]:
df_canada = pd.read_csv('canada_updated.csv')
df_canada["date"] = pd.to_datetime(df_canada["date"])
df_canada.head()

FileNotFoundError: [Errno 2] No such file or directory: 'canada_updated.csv'

## Copie pour modification

In [5]:
df_model = df_canada.copy()

df_model['date'] = pd.to_datetime(df_model['date'], errors='coerce')
df_model.sort_values(by=['cid', 'date'], inplace=True)

# Retirer les lignes où Quality_Flag est False
df_model = df_model[df_model['QUALITY_FLAG'] == True]

# (FACULTATIF) Exclure les banques
# df_model = df_model[df_model['industry'] != 'Banks']

## Selection de features

In [6]:
def select_features(
    df,
    include_agro=False,
    include_rgro=False,
    include_tcgro=False,
    include_ratios_assets=False,
    include_ratios_rev=False,
    include_ratios_totcap=False,
    mandatory_cols=None
):
    """
    Sélectionne dynamiquement les colonnes d'un DataFrame en fonction
    des familles de variables explicatives demandées,
    en plaçant d'abord les colonnes obligatoires (mandatory_cols).
    """
    
    # 1) Gérer la liste mandatory_cols (par défaut : vide ou ['cid','date'] selon besoin)
    if mandatory_cols is None:
        mandatory_cols = []
    
    # 2) Définir les "familles" de motifs
    family_patterns = {
        'agro': ['_agro_1q', '_agro_4q'],
        'rgro': ['_rgro_1q', '_rgro_4q'],
        'tcgro': ['_tcgro_1q', '_tcgro_4q'],
        'ratios_assets': ['_on_assets_ratio'],
        'ratios_rev': ['_on_rev_ratio'],
        'ratios_totcap': ['_on_tot_cap_ratio']
    }
    
    # 3) Construire la liste des motifs à inclure
    patterns_to_keep = []
    if include_agro:
        patterns_to_keep += family_patterns['agro']
    if include_rgro:
        patterns_to_keep += family_patterns['rgro']
    if include_tcgro:
        patterns_to_keep += family_patterns['tcgro']
    
    if include_ratios_assets:
        patterns_to_keep += family_patterns['ratios_assets']
    if include_ratios_rev:
        patterns_to_keep += family_patterns['ratios_rev']
    if include_ratios_totcap:
        patterns_to_keep += family_patterns['ratios_totcap']
    
    # 4) Retrouver toutes les colonnes du df qui matchent nos motifs
    matched_cols = set()
    for pat in patterns_to_keep:
        for col in df.columns:
            if pat in col:
                matched_cols.add(col)
    # => matched_cols est un set() de colonnes
    
    # 5) Conserver l'ordre original des colonnes matched, 
    #    en filtrant df.columns dans l'ordre d'origine
    matched_cols_in_order = [c for c in df.columns if c in matched_cols]
    
    # 6) Construire l'ordre final :
    #    - d'abord mandatory_cols (dans l'ordre donné),
    #    - puis les matched_cols (dans l'ordre d'origine)
    #    - attention aux colonnes obligatoires qui n'existent pas, 
    #      ou aux duplications
    #    - on fait donc une intersection + un set() pour éviter 
    #      les collisions.
    
    # Intersection pour ne pas inclure des mandatory inexistantes
    mandatory_cols_in_df = [c for c in mandatory_cols if c in df.columns]
    
    # Puis on concatène en évitant toute duplication
    columns_to_keep_ordered = mandatory_cols_in_df + [
        c for c in matched_cols_in_order if c not in mandatory_cols_in_df
    ]
    
    # 7) Extraire le sous-DataFrame
    df_filtered = df[columns_to_keep_ordered].copy()
    
    # 8) (Optionnel) trier par cid/date si elles sont présentes
    if 'cid' in df_filtered.columns and 'date' in df_filtered.columns:
        df_filtered.sort_values(by=['cid', 'date'], inplace=True)
    
    return df_filtered

## Appel de fonction - Features

In [7]:
df_model_final = select_features(
    df_model,
    include_agro=True,
    include_rgro=True,
    include_tcgro=True,
    include_ratios_assets=True,
    include_ratios_rev=True,
    include_ratios_totcap=True,
    mandatory_cols=['cid', 'date', 'binary_target_cash_operations']  # je garde la target
)

# Compter le nombre total de NaN dans tout le DataFrame
total_nan = df_model_final.isna().sum().sum()
print(f"Nombre total de valeurs NaN dans df_model_final : {total_nan}")

# Retirer les lignes qui contiennent AU MOINS un NaN
df_model_final.dropna(inplace=True)

# Vérifier à nouveau qu’il n’y a plus de NaN
total_nan_apres = df_model_final.isna().sum().sum()
print(f"Nombre total de valeurs NaN après suppression : {total_nan_apres}")

print(df_model_final.shape)

Nombre total de valeurs NaN dans df_model_final : 1360861
Nombre total de valeurs NaN après suppression : 0
(36154, 579)


## DF 10 ans

In [1]:
# Date la plus récente du DataFrame
max_date = df_model_final['date'].max()

# Date de coupure (5 ans avant)
cutoff_date = max_date - pd.DateOffset(years=10)

# Filtrer pour ne garder que les 5 dernières années
df_test = df_model_final[df_model_final['date'] >= cutoff_date].copy()

print(df_test['date'].min(), df_test['date'].max())
print(df_test.shape)

NameError: name 'df_model_final' is not defined

## Rolling Window + AutoML

In [9]:
def pipeline_rolling_windows(data, date_col, target_col, train_years, val_years, test_years, buffer_months=0):
    """
    Pipeline direct pour la rolling window avec AutoML et cross-validation personnalisée.
    Ajoute les périodes dans le DataFrame final pour validation.
    """
    # Conversion de la colonne date
    data[date_col] = pd.to_datetime(data[date_col])
    start_date = data[date_col].min()
    end_date = data[date_col].max()

    predictions_all = []  # Liste pour stocker toutes les prédictions

    while start_date + relativedelta(years=train_years + val_years + test_years) <= end_date:
        # Définir les périodes
        train_end = start_date + relativedelta(years=train_years) - pd.Timedelta(days=1)
        tampon_1_end = train_end + relativedelta(months=buffer_months)
        val_start = tampon_1_end + pd.Timedelta(days=1)
        val_end = val_start + relativedelta(years=val_years) - pd.Timedelta(days=1)
        tampon_2_end = val_end + relativedelta(months=buffer_months)
        test_start = tampon_2_end + pd.Timedelta(days=1)
        test_end = test_start + relativedelta(years=test_years) - pd.Timedelta(days=1)

        # Filtrer les données
        train_data = data.loc[(data[date_col] >= start_date) & (data[date_col] <= train_end)]
        val_data = data.loc[(data[date_col] >= val_start) & (data[date_col] <= val_end)]
        test_data = data.loc[(data[date_col] >= test_start) & (data[date_col] <= test_end)]

        if len(train_data) == 0 or len(val_data) == 0 or len(test_data) == 0:
            print(f"Fenêtre {start_date.year}-{test_end.year} : données insuffisantes, sautée.")
            start_date += relativedelta(years=1)
            continue

        # Configurer et entraîner AutoML
        print(f"Fenêtre {start_date.year}-{test_end.year} : entraînement de AutoML...")
        automl = AutoML(results_path=f"AutoML_{start_date.year}-{test_end.year}", mode="Perform", algorithms=["Xgboost"], eval_metric="auc")
        custom_cv = [(train_data.index, val_data.index)]
        automl.fit(
            train_data.drop(columns=[target_col, date_col, 'cid']),
            train_data[target_col], cv=custom_cv
        )

        # Prédire sur le test set
        test_preds = test_data[[date_col, target_col]].copy()
        #test_preds["predicted"] = automl.predict_proba(test_data.drop(columns=[target_col, date_col, 'cid']))
        proba = automl.predict_proba(test_data.drop(columns=[target_col, date_col, 'cid']))
        test_preds["prob_down"] = proba[:, 0]  # Probabilité pour la classe 0 (diminution des bénéfices)
        test_preds["prob_up"] = proba[:, 1]  # Probabilité pour la classe 1 (augmentation des bénéfices)
        test_preds["net_prob_score"] = test_preds["prob_up"] - test_preds["prob_down"]
        test_preds["window"] = f"{start_date.year}-{test_end.year}"
        test_preds["cid"] = test_data["cid"].values

        # Ajouter les périodes pour validation
        #test_preds["train_start"] = start_date
        #test_preds["train_end"] = train_end
        #test_preds["tampon_1"] = tampon_1_end
        #test_preds["val_start"] = val_start
        #test_preds["val_end"] = val_end
        #test_preds["tampon_2"] = tampon_2_end
        #test_preds["test_start"] = test_start
        #test_preds["test_end"] = test_end

        # Sauvegarder les prédictions
        predictions_all.append(test_preds)

        # Avancer la fenêtre
        start_date += relativedelta(years=1)

    predictions_df = pd.concat(predictions_all, ignore_index=True)
    return predictions_df

## Appel de fonction - Prédictions

In [10]:
predictions_df = pipeline_rolling_windows(
    data=df_model_final, 
    date_col="date", 
    target_col="binary_target_cash_operations", 
    train_years=2, 
    val_years=1, 
    test_years=1, 
    buffer_months=1
)

Fenêtre 2002-2006 : entraînement de AutoML...
AutoML directory: AutoML_2002-2006
The task is binary_classification with evaluation metric auc
AutoML will use algorithms: ['Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'ensemble']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 1 model
1_Default_Xgboost auc 0.876088 trained in 24.75 seconds (1-sample predict time 0.1374 seconds)
* Step not_so_random will try to check up to 4 models
2_Xgboost auc 0.873305 trained in 20.29 seconds (1-sample predict time 0.1451 seconds)
3_Xgboost auc 0.870281 trained in 20.47 seconds (1-sample predict time 0.1587 seconds)
4_Xgboost auc 0.857249 trained in 15.26 seconds (1-sample predict time 0.0909 seconds)
5_Xgboost auc 0.778136 trained in 10.1 seconds (1-sample pre

/Users/dominicprenovost/anaconda3/envs/HECFinance/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.


None 57
Add Golden Feature: E_G_unusual_items_on_tot_cap_ratio_sum_E_G_int_div_income_tcgro_1q
Add Golden Feature: E_G_unusual_items_on_tot_cap_ratio_sum_E_G_effective_tax_rate_on_assets_ratio
Add Golden Feature: E_G_restructuring_charges_tcgro_4q_sum_E_G_unusual_items_agro_4q
Add Golden Feature: E_G_net_income_inc_agro_4q_diff_E_G_ebit_tcgro_1q
Add Golden Feature: E_G_unusual_items_agro_4q_sum_E_G_restructuring_charges_agro_4q
Add Golden Feature: E_G_net_income_inc_tcgro_4q_diff_E_G_norm_net_inc_tcgro_4q
Add Golden Feature: E_G_unusual_items_agro_4q_diff_E_G_cost_borrowing_tcgro_1q
Add Golden Feature: E_G_minority_int_rgro_1q_diff_E_G_net_income_inc_on_tot_cap_ratio
Add Golden Feature: E_G_r_d_agro_4q_diff_E_G_unusual_items_on_assets_ratio
Add Golden Feature: E_G_divestitures_on_tot_cap_ratio_sum_E_G_unusual_items_agro_4q
Add Golden Feature: E_G_restructuring_charges_on_tot_cap_ratio_diff_E_G_unusual_items_on_tot_cap_ratio
Add Golden Feature: E_G_net_income_inc_on_tot_cap_ratio_sum_E_

In [11]:
predictions_df.to_csv("df_test_2025.csv", index=False)

## AUC - Overall

In [27]:
test_auc_net = roc_auc_score(predictions_df["binary_target_cash_operations"], predictions_df["net_prob_score"])
test_auc_up = roc_auc_score(predictions_df["binary_target_cash_operations"], predictions_df["prob_up"])
test_auc_down = roc_auc_score(predictions_df["binary_target_cash_operations"], predictions_df["prob_down"])
test_auc_net, test_auc_up, test_auc_down

(0.6481474991605236, 0.6481475031998658, 0.3518524988198052)

## Grouped AUCs

In [12]:
def compute_auc_by_groups(
    predictions_df: pd.DataFrame, 
    df_canada: pd.DataFrame,
    group_cols: list,
    cid_col: str = "cid",
    date_col: str = "date",
    target_col: str = "binary_target_cash_operations",
    prob_col: str = "prob_up",
    out_csv: str = None
):
    """
    Calcule l'AUC par groupes (selon group_cols) et le nombre de lignes dans chaque groupe.
    
    Arguments :
    -----------
    - predictions_df : DataFrame contenant au moins [cid_col, date_col, target_col, prob_col]
    - df_canada : DataFrame contenant au moins [cid_col, date_col, 'return_1q', 'industry_raw', etc.]
    - group_cols : liste des colonnes sur lesquelles faire le groupby (ex: ["year", "industry_raw"])
    - cid_col : nom de la colonne identifiant l'entreprise (défaut: "cid")
    - date_col : nom de la colonne date (défaut: "date")
    - target_col : nom de la colonne cible binaire (ex: 0/1) (défaut: "binary_target_cash_operations")
    - prob_col : nom de la colonne contenant la probabilité pour la classe positive (défaut: "prob_up")
    - out_csv : chemin vers le fichier CSV de sortie (défaut: None, ne pas exporter)

    Retour :
    --------
    - final_df : DataFrame avec AUC et Count pour chaque groupe
    """

    # 1) Fusionner les dataframes (left merge pour conserver toutes les prédictions)
    df_merged = predictions_df.merge(
        df_canada[[cid_col, date_col, 'return_1q', 'industry_raw']],  # adapt si tu veux plus/moins de colonnes
        on=[cid_col, date_col],
        how='left'
    )

    # 2) Si "year" est dans group_cols et pas encore créée, on l'extrait du date_col
    if "year" in group_cols and "year" not in df_merged.columns:
        df_merged["year"] = pd.to_datetime(df_merged[date_col]).dt.year

    # -- Fonction interne pour calculer l’AUC sur un sous-groupe
    def group_auc(sub_df):
        unique_targets = sub_df[target_col].unique()
        # Gérer le cas où il n’y a qu’une seule classe (roc_auc_score plantait sinon)
        if len(unique_targets) < 2:
            return float("nan")
        return roc_auc_score(sub_df[target_col], sub_df[prob_col])

    # 3) Calculer l’AUC par groupe
    grouped_aucs = (
        df_merged
        .groupby(group_cols)
        .apply(group_auc)
        .reset_index(name="AUC")
    )

    # 4) Calculer le nombre d’observations par groupe (ou nombre de cid uniques, selon besoin)
    #    Ici : on compte simplement le nombre de lignes
    grouped_counts = (
        df_merged
        .groupby(group_cols)[target_col]  # ou [cid_col] si tu veux le nombre d'entreprises distinctes
        .count()
        .reset_index(name="Count")
    )

    # 5) Fusionner AUC et Count
    final_df = pd.merge(grouped_aucs, grouped_counts, on=group_cols, how="left")

    # Optionnel : trier le résultat
    final_df = final_df.sort_values(group_cols)

    # Optionnel : exporter en CSV
    if out_csv is not None:
        final_df.to_csv(out_csv, index=False)

    return final_df

In [13]:
compute_auc_by_groups(
    predictions_df=predictions_df,
    df_canada=df_canada,
    group_cols=["year", "industry_raw"],
    out_csv="auc_par_annee_et_industrie.csv"
)

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1301/2025299659.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


,year,industry_raw,AUC,Count
0,2005,Aerospace & Defense,0.833333,7
1,2005,Automobile Components,0.142857,8
2,2005,Beverages,0.333333,7
3,2005,Biotechnology,0.757576,17
4,2005,Broadline Retail,0.600000,12
...,...,...,...,...
1101,2024,Specialty Retail,NaN,12
1102,2024,"Textiles, Apparel & Luxury Goods",NaN,4
1103,2024,Trading Companies & Distributors,NaN,15
1104,2024,Transportation Infrastructure,NaN,2


In [14]:
compute_auc_by_groups(
    predictions_df=predictions_df,
    df_canada=df_canada,
    group_cols=["industry_raw"],
    out_csv="CFOP_auc_par_industrie.csv"
)

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1301/2025299659.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


,industry_raw,AUC,Count
0,Aerospace & Defense,0.626663,314
1,Air Freight & Logistics,0.605128,49
2,Automobile Components,0.578946,337
3,Beverages,0.573076,179
4,Biotechnology,0.632759,249
...,...,...,...
69,Tobacco,0.875817,26
70,Trading Companies & Distributors,0.553094,535
71,Transportation Infrastructure,0.590618,123
72,Undefined,0.765432,48


In [15]:
compute_auc_by_groups(
    predictions_df=predictions_df,
    df_canada=df_canada,
    group_cols=["year"],
    out_csv="CFO_auc_par_annee.csv"
)

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1301/2025299659.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


,year,AUC,Count
0,2005,0.583537,975
1,2006,0.659689,2014
2,2007,0.676335,2109
3,2008,0.675696,1959
4,2009,0.741570,1847
5,2010,0.616945,1726
6,2011,0.638033,1726
7,2012,0.699479,1715
8,2013,0.643880,1671
9,2014,0.681990,1594


## Création de Portefeuille + Annual Weighted Returns

In [16]:
def create_weighted_portfolios(predictions_df, df_canada, proba_col, return_col, lower_threshold=0.4, upper_threshold=0.6):
    """
    Crée un portefeuille pondéré basé sur les prédictions et calcule les rendements pondérés par année,
    en attribuant des poids positifs aux positions longues et négatifs aux positions courtes, avec une somme neutre.
    
    Args:
    - predictions_df (pd.DataFrame): DataFrame contenant les prédictions et les identifiants 'cid'.
    - df_canada (pd.DataFrame): DataFrame contenant les rendements futurs et les identifiants 'cid'.
    - return_col (str): Nom de la colonne des rendements futurs dans df_canada.
    - lower_threshold (float): Seuil inférieur pour les positions courtes.
    - upper_threshold (float): Seuil supérieur pour les positions longues.
    
    Returns:
    - result_df (pd.DataFrame): DataFrame contenant les rendements pondérés des portefeuilles par année.
    """
    # Assure que les colonnes 'date' sont au bon format datetime dans les deux DataFrames
    predictions_df['date'] = pd.to_datetime(predictions_df['date'])
    df_canada['date'] = pd.to_datetime(df_canada['date'])
        
    
    # Joindre les deux DataFrames sur 'cid' et 'date'
    merged_df = predictions_df.merge(df_canada[['cid', 'date', return_col]], on=['cid', 'date'], how='left')
    
    # Extraire l'année à partir de la colonne 'date'
    merged_df['year'] = merged_df['date'].dt.year
    
    # Retirer les lignes avec des valeurs manquantes
    merged_df.dropna(inplace=True)
    
    # Initialiser une liste pour stocker les résultats
    results = []

    # Grouper par année
    for year, group in merged_df.groupby('year'):
        # Sélectionner les positions longues et courtes selon les seuils
        selected_long = group[group[proba_col] > upper_threshold]  # Positions longues
        selected_short = group[group[proba_col] < lower_threshold]  # Positions courtes
        
        n_long = len(selected_long)
        n_short = len(selected_short)
        
        if n_long > 0 or n_short > 0:
            # Attribuer des poids aux positions longues et courtes
            if n_long > 0:
                selected_long.loc[:, 'weight'] = 1 / n_long  # Poids positifs pour les positions longues
            if n_short > 0:
                selected_short.loc[:, 'weight'] = -1 / n_short  # Poids négatifs pour les positions courtes
            
            # Combiner les deux DataFrames
            selected = pd.concat([selected_long, selected_short], ignore_index=True)
            
            # Calculer le rendement pondéré du portefeuille (somme des rendements pondérés)
            weighted_return = (selected['weight'] * selected[return_col] * 100).sum()
            
            # Ajouter le résultat à la liste
            results.append({'year': year, 'weighted_return': weighted_return})
        else:
            # Si aucune action ne respecte les seuils, le rendement est NaN
            results.append({'year': year, 'weighted_return': float('nan')})

    # Convertir les résultats en DataFrame
    result_df = pd.DataFrame(results)
    
    return result_df

## Appel de fonction rendement annuel

In [17]:
ret_net_income = create_weighted_portfolios(predictions_df, df_canada, 'prob_up', 'return_1q', lower_threshold=0.5, upper_threshold=0.5)

#ret_net_income.to_csv("returns_net_income_0.5_0.5.csv", index=False)

ret_net_income

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1301/444120012.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1301/444120012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1301/444120012.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,year,weighted_return
0,2005,-2.044758
1,2006,-1.734227
2,2007,0.169548
3,2008,5.246891
4,2009,17.606542
5,2010,-4.169367
6,2011,-8.712187
7,2012,1.407976
8,2013,-7.407460
9,2014,1.500819


In [18]:
returns_net_income = create_weighted_portfolios(predictions_df, df_canada, 'prob_up', 'return_1q', lower_threshold=0.4, upper_threshold=0.6)

#returns_net_income.to_csv("returns_net_income_0.4_0.6.csv", index=False)

returns_net_income

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1301/444120012.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1301/444120012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1301/444120012.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,year,weighted_return
0,2005,-2.621053
1,2006,-1.792627
2,2007,-1.263394
3,2008,5.240312
4,2009,19.754888
5,2010,-4.883111
6,2011,-8.608529
7,2012,0.630128
8,2013,-8.029594
9,2014,2.132954


## Rendement par décile

In [19]:
def compute_decile_returns(
    predictions_df: pd.DataFrame,
    df_canada: pd.DataFrame,
    proba_col: str,
    return_col: str,
    aggregator: str = "mean",        # "mean" ou "median"
    decile_weights: list = None,     # liste de 10 poids ou None
    csv_path: str = None             # chemin CSV ou None
) -> pd.DataFrame:
    """
    Calcule le rendement par décile (du plus faible au plus élevé) pour chaque année.
    - Découpage en déciles via pd.qcut (q=10).
    - Agrégation des rendements par décile via un 'aggregator' (mean ou median).
    - Optionnellement, applique un vecteur de poids (decile_weights) pour calculer
      un rendement global (somme pondérée des déciles).

    Paramètres
    ----------
    predictions_df : pd.DataFrame
        Contient les prédictions (probabilités) et les colonnes 'cid', 'date'.
        Doit inclure la colonne `proba_col`.
    df_canada : pd.DataFrame
        Contient la colonne `return_col` ainsi que 'cid', 'date'.
    proba_col : str
        Nom de la colonne dans predictions_df représentant la probabilité ou le score.
    return_col : str
        Nom de la colonne dans df_canada qui contient le rendement (ou variation).
    aggregator : {"mean", "median"}, défaut = "mean"
        Choix entre la moyenne ou la médiane comme statistique de rendement pour le décile.
    decile_weights : list of float (length 10) ou None
        Si fourni, liste de 10 poids (un pour chaque décile de 0 à 9).
        Permet de calculer une colonne "portfolio_return" = somme(décile_i * poids_i).
    csv_path : str ou None
        Si un chemin est fourni, le DataFrame final sera sauvegardé en CSV.

    Retour
    ------
    df_final : pd.DataFrame
        DataFrame pivoté avec:
        - 1 ligne par année
        - 10 colonnes (D1 à D10) indiquant le rendement agrégé de chaque décile
        - Si decile_weights est fourni, une colonne supplémentaire 'portfolio_return'
          représentant la somme pondérée des rendements déciles.
    """

    # --- 1) Merge sur cid + date
    merged_df = predictions_df.merge(
        df_canada[['cid', 'date', return_col]],
        on=['cid', 'date'],
        how='left'
    )
    
    # --- 2) Extraire l'année
    merged_df['year'] = merged_df['date'].dt.year
    
    # --- 3) Nettoyage éventuel
    merged_df.dropna(subset=[proba_col, return_col, 'year'], inplace=True)
    
    # --- 4) Choix de la fonction d'agrégation
    if aggregator == "mean":
        aggregator_func = np.mean
    elif aggregator == "median":
        aggregator_func = np.median
    else:
        raise ValueError("aggregator must be 'mean' or 'median'.")
    
    # --- 5) Préparer une liste pour stocker (year, decile_bin, agg_return)
    decile_results = []
    
    for year, group in merged_df.groupby('year'):
        # On doit découper en 10 déciles
        group = group.copy()
        
        # Astuce: labels=False renvoie des labels 0..9
        group['decile_bin'] = pd.qcut(group[proba_col], q=10, labels=False)
        
        # Calculer l'agrégat (mean ou median) du rendement dans chaque décile
        for decile_id, subdf in group.groupby('decile_bin'):
            if pd.isnull(decile_id):
                continue
            
            decile_id = int(decile_id)
            # Rendement agrégé (en % si on veut)
            decile_value = aggregator_func(subdf[return_col]) * 100.0
            
            decile_results.append({
                'year': year,
                'decile': decile_id,
                'decile_return': decile_value
            })
    
    # --- 6) On transforme la liste en DataFrame + pivot
    decile_df = pd.DataFrame(decile_results)
    df_pivot = decile_df.pivot(index='year', columns='decile', values='decile_return')
    
    # Renommer les colonnes pour D1..D10
    # Attention : decile_id = 0 correspond au 1er décile => D1
    df_pivot.columns = [f"D{i+1}" for i in df_pivot.columns]
    
    # --- 7) Si on a des poids, on calcule la somme pondérée
    if decile_weights is not None:
        if len(decile_weights) != 10:
            raise ValueError("decile_weights must be a list of length 10.")
        
        # Multiplier chaque colonne D1..D10 par le poids correspondant, puis sommer
        # Astuce : df_pivot.iloc[:, 0:10] prend les 10 premières colonnes => D1..D10
        # On peut ensuite .mul(decile_weights, axis='columns') pour multiplier par le vecteur
        # et .sum(axis=1) pour obtenir la somme par année
        # (on suppose que l'ordre des colonnes est D1..D10).
        
        weighted_sum = df_pivot.iloc[:, 0:10].mul(decile_weights, axis='columns').sum(axis=1)
        
        # On l'ajoute comme nouvelle colonne
        df_pivot['portfolio_return'] = weighted_sum
    
    # --- 8) On remet l'index 'year' comme colonne
    df_final = df_pivot.reset_index()
    
    # --- 9) Sauvegarde CSV si demandé
    if csv_path is not None:
        df_final.to_csv(csv_path, index=False)
    
    return df_final

In [29]:
df_deciles = compute_decile_returns(
    predictions_df=predictions_df,
    df_canada=df_canada,
    proba_col='prob_up',
    return_col='return_1q',
    aggregator='mean',
    decile_weights=[-0.25, -0.25, -0.25, -0.25, 0, 0, 0.25, 0.25, 0.25, 0.25],
    csv_path='CFO_my_decile_returns.csv'
)

df_deciles.head()

,year,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,portfolio_return
0,2005,11.323382,12.795672,9.385665,10.279796,8.249730,9.821087,12.206950,6.498888,10.884432,6.636068,-1.889544
1,2006,3.446530,6.007244,6.296100,3.610876,3.187303,2.704650,3.797558,4.720456,5.300471,3.370731,-0.542883
2,2007,12.232343,7.955634,6.411043,6.482127,8.356652,8.566385,3.103633,7.010022,6.256700,6.141823,-2.642242
3,2008,-25.009816,-21.572715,-13.032412,-16.642532,-12.232140,-14.730729,-5.624056,-7.986846,-10.737167,-7.887927,11.005370
4,2009,10.528919,15.558400,15.060952,13.081065,18.982264,19.442698,19.519046,19.082624,53.791485,27.048754,16.303143


## Preuve CSV Rendement par Décile

In [21]:
# --- 1) Merge sur cid + date
merged_df = predictions_df.merge(
    df_canada[['cid', 'date', 'return_1q']],
    on=['cid', 'date'],
    how='left'
    )

# --- 2) Extraire l'année
merged_df['year'] = merged_df['date'].dt.year

# Filtrer pour l'année 2023
year = 2023
year_df = merged_df[merged_df['year'] == year].copy()

# Attribuer les déciles pour cette année
year_df['decile_bin'] = pd.qcut(year_df['prob_up'], q=10, labels=False)

# Liste pour stocker chaque subdf avec une colonne indiquant le décile
list_of_subdfs = []

# Itérer sur chaque décile et collecter les sous-ensembles
for decile_id, subdf in year_df.groupby('decile_bin'):
    # Ajouter une colonne indiquant le décile courant
    subdf = subdf.copy()
    subdf['decile_id'] = decile_id
    list_of_subdfs.append(subdf)

# Concaténer tous les subdf en un seul DataFrame
all_subdfs_df = pd.concat(list_of_subdfs, ignore_index=True)

# Exporter vers CSV
#all_subdfs_df.to_csv("subdfs_2023.csv", index=False)